In [ ]:
%cd "C:\\Users\\Administrator\\Desktop\\强化学习\\中证50指数增强\\"
path = "C:\\Users\\Administrator\\Desktop\\强化学习\\中证50指数增强\\"

import warnings
warnings.filterwarnings("ignore")
import time
import numpy as np
import pandas as pd
#显示Dateframe所有列(参数设置为None代表显示所有行，也可以自行设置数字)
pd.set_option('display.max_columns',None)
#显示Dateframe所有行
pd.set_option('display.max_rows',None)
#设置Dataframe数据的显示长度，默认为50
pd.set_option('max_colwidth',None)
#不显示科学计数法，保留5位小数
pd.set_option('display.float_format', lambda x: '%.5f' % x)

import datetime
import os
from tqdm import tqdm
import baostock as bs

## Check normal stock data length (training data)


In [ ]:
train_start = '2018-01-01'
train_end = '2023-06-01'
test_start = '2023-01-01'
test_end = str(datetime.date.today())
#### 登陆系统 ####
lg = bs.login()
#### 获取沪深A股历史K线数据 ####
rs = bs.query_history_k_data_plus("sh.600000",
    "date, code, open, high, low, close, volume, amount, turn, pctChg",
    start_date=train_start, end_date=train_end,
    frequency="d", adjustflag="1")
#### 打印结果集 ####
data_list = []
while (rs.error_code == '0') & rs.next():
    # 获取一条记录，将记录合并在一起
    data_list.append(rs.get_row_data())
result = pd.DataFrame(data_list, columns=rs.fields)
train_standard_len = len(result)

train_standard_len

## Get index constituent stocks


In [ ]:
# 获取上证50成分股
rs = bs.query_sz50_stocks()
# 打印结果集
sz50_stocks = []
while (rs.error_code == '0') & rs.next():
    # 获取一条记录，将记录合并在一起
    sz50_stocks.append(rs.get_row_data())
result = pd.DataFrame(sz50_stocks, columns=rs.fields)

result.shape, result.head(5)

## Get component data (training data)

In [ ]:
pre_stock_pool = result.code.to_list()
stock_pool = []
train_df = pd.DataFrame()
for i in tqdm(range(len(pre_stock_pool))):
    tmp_stock = pre_stock_pool[i]
    rs = bs.query_history_k_data_plus(tmp_stock,
    "date, code, open, high, low, close, volume, amount, turn, pctChg, peTTM, pbMRQ, psTTM, pcfNcfTTM",
    start_date=train_start, end_date=train_end,
    frequency="d", adjustflag="1")
    #### 打印结果集 ####
    data_list = []
    while (rs.error_code == '0') & rs.next():
        # 获取一条记录，将记录合并在一起
        data_list.append(rs.get_row_data())
    tmp_result = pd.DataFrame(data_list, columns=rs.fields)
    if len(tmp_result) != train_standard_len:
        print('Skipping: ', tmp_stock, ' due to insufficient data length')
    else:
        train_df = train_df.append(tmp_result)
        stock_pool.append(tmp_stock)
    time.sleep(1)
train_df.to_csv('数据//train_df.csv')

len(stock_pool), train_df.shape, train_df.head(5)

## Get component data (test data)

In [ ]:
test_df = pd.DataFrame()
for i in tqdm(range(len(stock_pool))):
    tmp_stock = stock_pool[i]
    rs = bs.query_history_k_data_plus(tmp_stock,
    "date, code, open, high, low, close, volume, amount, turn, pctChg, peTTM, pbMRQ, psTTM, pcfNcfTTM",
    start_date=test_start, end_date=test_end,
    frequency="d", adjustflag="1")
    #### 打印结果集 ####
    data_list = []
    while (rs.error_code == '0') & rs.next():
        # 获取一条记录，将记录合并在一起
        data_list.append(rs.get_row_data())
    tmp_result = pd.DataFrame(data_list, columns=rs.fields)
    test_df = test_df.append(tmp_result)
    time.sleep(1)
test_df.to_csv('数据//test_df.csv')

test_df.shape, test_df.head(5)